In [1]:
import cv2
import gymnasium
import os
from gymnasium.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete
import numpy as np
import random

In [2]:
import cv2
import gymnasium
import os
from gym import Env
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor
from gymnasium.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete
import numpy as np
import random
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO,DQN
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [3]:
maze=np.array([
    [ 1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
    [ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  1.,  1.],
    [ 1.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1.],
    [ 1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.],
    [ 1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.]
], dtype=np.int32)

In [4]:
import numpy as np
import cv2
import gymnasium

class MazeSolver(gymnasium.Env):
    def __init__(self, maze, target, rat=(0, 0)):
        self.og_rat = rat
        self.path = []
        self.rat = rat
        self.target = target
        self.tr = 0
        self.info = {}
        self.action_space = gymnasium.spaces.Discrete(4)
        self.maze = maze
        self.height, self.width = maze.shape
        self.observation_space = gymnasium.spaces.Box(low=0, high=1, shape=(self.height, self.width, 2), dtype=np.int16)

    def reset(self, **kwargs):
        self.rat = self.og_rat
        self.path = []
        self.tr = 0
        return (self._get_observation(),self.info)
    
    def step(self, action):
        prev = self.rat
        done = False
        reward = 0
        
        if action == 0:
            self.rat = (self.rat[0] - 1, self.rat[1])
        elif action == 1:
            self.rat = (self.rat[0], self.rat[1] + 1)
        elif action == 2:
            self.rat = (self.rat[0] + 1, self.rat[1])
        else:
            self.rat = (self.rat[0], self.rat[1] - 1)
            
        if self.rat[0] < 0 or self.rat[1] < 0 or self.rat[0] >= self.height or self.rat[1] >= self.width:
            reward += -0.8
            self.rat = prev 
            print("Hit a boundry")
        elif self.maze[self.rat[0]][self.rat[1]] == 0:
            reward += -0.8
            print("Hit obstacle")
            self.rat = prev
        elif self.rat in self.path:
            reward += -0.25
        elif self.maze[self.rat[0]][self.rat[1]] == 0:
            reward += -0.04
        elif self.rat == self.target:
            reward += 1.0
            print("target found")
            done = True
            
        self.tr += reward
        if self.tr < (-0.5 * self.height * self.width):
            print("Overtime")
            done = True
            
        self.path.append(self.rat)
        self.info = {"TimeLimit.truncated": False}
        return self._get_observation(), reward, done, False, self.info

    def _get_observation(self):
        rat_pos = np.zeros((self.height, self.width), dtype=np.int16)
        rat_pos[self.rat[0], self.rat[1]] = 1
        observation = np.dstack((self.maze, rat_pos))
        return observation.astype(np.int16)
    def render(self, mode='human'):
        canvas = np.zeros((self.height, self.width, 3), dtype=np.uint8)
        canvas[:, :, 2] = self.maze * 255  
        canvas[self.rat[0], self.rat[1], :] = [0, 255, 0]  
        canvas[self.target[0], self.target[1], :] = [255, 0, 0] 
        resized_canvas = cv2.resize(canvas, None, fx=50, fy=50, interpolation=cv2.INTER_NEAREST)
        cv2.imshow('Maze', resized_canvas)
        cv2.waitKey(100)

In [5]:
env=MazeSolver(maze,(9,9))
check_env(env)

Hit a boundry
Hit a boundry
Hit a boundry
Hit a boundry


/Users/janavbhasin/miniconda3/lib/python3.10/site-packages/stable_baselines3/common/env_checker.py:30: UserWarning: It seems that your observation  is an image but its `dtype` is (int16) whereas it has to be `np.uint8`. If your observation is not an image, we recommend you to flatten the observation to have only a 1D vector
  warnings.warn(
/Users/janavbhasin/miniconda3/lib/python3.10/site-packages/stable_baselines3/common/env_checker.py:38: UserWarning: It seems that your observation space  is an image but the upper and lower bounds are not in [0, 255]. Because the CNN policy normalize automatically the observation you may encounter issue if the values are not in that range.
  warnings.warn(
/Users/janavbhasin/miniconda3/lib/python3.10/site-packages/stable_baselines3/common/env_checker.py:51: UserWarning: The minimal resolution for an image is 36x36 for the default `CnnPolicy`. You might need to use a custom features extractor cf. https://stable-baselines3.readthedocs.io/en/master/gui

In [9]:
model = PPO('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=500000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Hit obstacle
Hit a boundry
Hit obstacle
Hit a boundry
Hit obstacle
Hit obstacle
Hit obstacle
Hit a boundry
Hit a boundry
Hit a boundry
Hit a boundry
Hit a boundry
Hit a boundry
Hit obstacle
Hit a boundry
Hit a boundry
Hit obstacle
Hit obstacle
Hit obstacle
Hit obstacle
Hit obstacle
Hit a boundry
Hit obstacle
Hit obstacle
Hit a boundry
Hit a boundry
Hit obstacle
Hit a boundry
Hit obstacle
Hit obstacle
Hit a boundry
Hit obstacle
Hit obstacle
Hit obstacle
Hit obstacle
Hit a boundry
Hit a boundry
Hit obstacle
Hit a boundry
Overtime
Hit a boundry
Hit a boundry
Hit a boundry
Hit obstacle
Hit a boundry
Hit a boundry
Hit a boundry
Hit a boundry
Hit a boundry
Hit obstacle
Hit obstacle
Hit a boundry
Hit obstacle
Hit obstacle
Hit a boundry
Hit a boundry
Hit a boundry
Hit a boundry
Hit a boundry
Hit obstacle
Hit a boundry
Hit a boundry
Hit a boundry
Hit a boundry
Hit a boundry
Hit a boundry
Hit a boundry


In [10]:
episodes = 1
for episode in range(1, episodes+1):
    obs,_ = env.reset()
    done = False
    score = 0 
    while not done:
        action, _states = model.predict(obs)
        obs, reward, done,_, info = env.step(action)
        score+=reward
        env.render()
    print('Episode:{} Score:{}'.format(episode, score))
    print(env.path)
    cv2.destroyAllWindows()

Hit obstacle
Hit obstacle
Hit obstacle
Overtime
Episode:1 Score:-50.15
[(1, 0), (1, 1), (1, 2), (1, 3), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 9), (1, 8), (1, 7), (2, 7), (3, 7), (3, 8), (3, 9), (4, 9), (5, 9), (5, 8), (5, 7), (5, 6), (5, 5), (5, 5), (5, 5), (5, 5), (6, 5), (6, 4), (6, 3), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), (7, 3), (7, 4), 

In [11]:
class UCS:
    def __init__(self, maze):
        self.maze = maze
        self.height = len(maze)
        self.width = len(maze[0])
    
    def get_children(self, current):
        children = []
        x, y = current
        moves = [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]
        for move in moves:
            if 0 <= move[0] < self.maze.shape[0] and 0 <= move[1] < self.maze.shape[1] and self.maze[move[0], move[1]] == 1:
                children.append(move)
        return children

    def ucs(self, start, goal):
        self.rat=start
        self.target=goal
        priority_queue = [(0, (start,[start]))]
        visited = set()
        path = []
        min_path=[]
        lminpath=float('inf')
        while priority_queue:
            priority_queue.sort()
            current_cost, (current_node,path) = priority_queue.pop(0)
            if len(path)>lminpath:
                continue
            if current_node == goal:
                lminpath=len(path)
                min_path=path
                cv2.destroyAllWindows()
                return path 
            children = self.get_children(current_node)
            for child in children:
                if child not in visited:
                    priority_queue.append((current_cost + 1, (child , path+[child])))
                    visited.add(child)
            self.rat=current_node
            self.render()
        cv2.destroyAllWindows()
        return None
    
    def render(self, mode='human'):
        canvas = np.zeros((self.height, self.width, 3), dtype=np.uint8)
        canvas[:, :, 2] = self.maze * 255
        canvas[self.rat[0], self.rat[1], :] = [0, 255, 0]  
        canvas[self.target[0], self.target[1], :] = [255, 0, 0] 
        resized_canvas = cv2.resize(canvas, None, fx=75, fy=75, interpolation=cv2.INTER_NEAREST)
        cv2.imshow('Maze', resized_canvas)
        cv2.waitKey(50)
    def render_path(self,path):
        for node in path:
            self.render()
            self.rat=node
        cv2.destroyAllWindows()

        
maze1=np.array([
    [ 1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
    [ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  1.,  1.],
    [ 1.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1.],
    [ 1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.],
    [ 1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.]
], dtype=np.int32)

rat=(0,0)
target=(9,9)

ucs= UCS(maze1)
path=ucs.ucs(rat,target)

print(path)
ucs.render_path(path)

[(0, 0), (1, 0), (1, 1), (1, 2), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 9), (2, 9), (3, 9), (4, 9), (5, 9), (5, 8), (5, 7), (5, 6), (5, 5), (6, 5), (6, 4), (6, 3), (6, 2), (6, 1), (6, 0), (7, 0), (8, 0), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (8, 6), (8, 7), (8, 8), (8, 9), (9, 9)]


In [12]:
class A_Star:
    def __init__(self, maze):
        self.maze = maze
        self.height=maze.shape[0]
        self.width=maze.shape[1]
    
    def get_children(self, current):
        children = []
        x, y = current
        moves = [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]
        for move in moves:
            if 0 <= move[0] < self.maze.shape[0] and 0 <= move[1] < self.maze.shape[1] and self.maze[move[0], move[1]] == 1:
                children.append(move)
        return children
    
    def heuristic(self, current, goal):
        return abs(current[0] - goal[0]) + abs(current[1] - goal[1])
    
    def a_star(self, start, goal):
        self.rat=start
        self.target=goal
        priority_queue = [(0, 0,(start,[start]))]
        visited = set(start)
        path = []
        min_path=[]
        lminpath=float('inf')
        while priority_queue:
            priority_queue.sort()
            current_cost,current_heuristic ,(current_node,path) = priority_queue.pop(0)
            if len(path)>lminpath:
                continue
            if current_node == goal:
                lminpath=len(path)
                min_path=path
                cv2.destroyAllWindows()
                return path 
            children = self.get_children(current_node)
            for child in children:
                if child not in visited:
                    priority_queue.append((current_cost+1+self.heuristic(child,goal)-current_heuristic,self.heuristic(child,goal),(child,path+[child])))
                    visited.add(child)
            self.rat=current_node
            self.render()
        cv2.destroyAllWindows()
        return None
    def render(self, mode='human'):
        canvas = np.zeros((self.height, self.width, 3), dtype=np.uint8)
        canvas[:, :, 2] = self.maze * 255
        canvas[self.rat[0], self.rat[1], :] = [0, 255, 0]  
        canvas[self.target[0], self.target[1], :] = [255, 0, 0] 
        resized_canvas = cv2.resize(canvas, None, fx=75, fy=75, interpolation=cv2.INTER_NEAREST)
        cv2.imshow('Maze', resized_canvas)
        cv2.waitKey(50)

    def render_path(self,path):
        for node in path:
            self.render()
            self.rat=node
        cv2.destroyAllWindows()

maze1=np.array([
    [ 1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
    [ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  1.,  1.],
    [ 1.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1.],
    [ 1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.],
    [ 1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.]
], dtype=np.int32)

rat=(0,0)
target=(9,9)


solver= A_Star(maze1)
path=solver.a_star(rat,target)

print(path)
solver.render_path(path)


[(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 9), (2, 9), (3, 9), (4, 9), (5, 9), (6, 9), (6, 8), (6, 7), (6, 6), (6, 5), (7, 5), (7, 4), (7, 3), (7, 2), (7, 1), (7, 0), (8, 0), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (8, 6), (8, 7), (8, 8), (8, 9), (9, 9)]


In [13]:
class Best_Frist:
    def __init__(self,maze):
        self.maze = maze
        self.height=maze.shape[0]
        self.width=maze.shape[1]

    def get_children(self, current):
        children = []
        x, y = current
        moves = [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]
        for move in moves:
            if 0 <= move[0] < self.maze.shape[0] and 0 <= move[1] < self.maze.shape[1] and self.maze[move[0], move[1]] == 1:
                children.append(move)
        return children
    
    def heuristic(self, current, goal):
        return abs(current[0] - goal[0]) + abs(current[1] - goal[1])

    def best_first(self, start, goal):
        self.rat=start
        self.target=goal
        priority_queue = [(0,(start,[start]))]
        visited = set(start)
        path = []
        min_path=[]
        lminpath=float('inf')
        while priority_queue:
            priority_queue.sort()
            current_heuristic ,(current_node,path) = priority_queue.pop(0)
            if len(path)>lminpath:
                continue
            if current_node == goal:
                lminpath=len(path)
                min_path=path
                cv2.destroyAllWindows()
                return path 
            children = self.get_children(current_node)
            for child in children:
                if child not in visited:
                    priority_queue.append((self.heuristic(child,goal),(child,path+[child])))
                    visited.add(child)
            self.rat=current_node
            self.render()
        cv2.destroyAllWindows()
        return None

    def render(self, mode='human'):
        canvas = np.zeros((self.height, self.width, 3), dtype=np.uint8)
        canvas[:, :, 2] = self.maze * 255
        canvas[self.rat[0], self.rat[1], :] = [0, 255, 0]  
        canvas[self.target[0], self.target[1], :] = [255, 0, 0] 
        resized_canvas = cv2.resize(canvas, None, fx=75, fy=75, interpolation=cv2.INTER_NEAREST)
        cv2.imshow('Maze', resized_canvas)
        cv2.waitKey(50)

    def render_path(self,path):
        for node in path:
            self.render()
            self.rat=node
        cv2.destroyAllWindows()

maze1=np.array([
    [ 1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
    [ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  1.,  1.],
    [ 1.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1.],
    [ 1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.],
    [ 1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.]
], dtype=np.int32)

rat=(0,0)
target=(9,9)

solver= Best_Frist(maze1)
path=solver.best_first(rat,target)

print(path)
solver.render_path(path)

[(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 9), (2, 9), (3, 9), (4, 9), (5, 9), (6, 9), (6, 8), (6, 7), (6, 6), (6, 5), (7, 5), (7, 4), (7, 3), (7, 2), (7, 1), (7, 0), (8, 0), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (8, 6), (8, 7), (8, 8), (8, 9), (9, 9)]


In [14]:
class BFS:
    def __init__(self,maze):
        self.maze=maze
        self.height=maze.shape[0]
        self.width=maze.shape[1]


    def get_children(self, current):
        children = []
        x, y = current
        moves = [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]
        for move in moves:
            if 0 <= move[0] < self.maze.shape[0] and 0 <= move[1] < self.maze.shape[1] and self.maze[move[0], move[1]] == 1:
                children.append(move)
        return children

    def bfs(self,start,goal):
        self.rat=start
        self.target=goal
        queue=[(start,[start])]
        visited = set(start)
        path = []
        min_path=[]
        lminpath=float('inf')
        while queue:
            current_node,path=queue.pop(0)
            if len(path)>lminpath:
                continue
            if current_node == goal:
                lminpath=len(path)
                min_path=path
                cv2.destroyAllWindows()
                return path 
            children = self.get_children(current_node)
            for child in children:
                if child not in visited:
                    queue.append((child,path+[child]))
                    visited.add(child)
            self.rat=current_node
            self.render()
        cv2.destroyAllWindows()
        return None

    def change_maze(self,path):
        for node in path:
            self.maze[node[0]][node[1]]=2
        return self.maze
    
    def render(self, mode='human'):
        canvas = np.zeros((self.height, self.width, 3), dtype=np.uint8)
        canvas[:, :, 2] = self.maze * 255
        canvas[self.rat[0], self.rat[1], :] = [0, 255, 0]  
        canvas[self.target[0], self.target[1], :] = [255, 0, 0] 
        resized_canvas = cv2.resize(canvas, None, fx=75, fy=75, interpolation=cv2.INTER_NEAREST)
        cv2.imshow('Maze', resized_canvas)
        cv2.waitKey(50)

    def render_path(self,path):
        for node in path:
            self.render()
            self.rat=node
        cv2.destroyAllWindows()

maze1=np.array([
    [ 1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
    [ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  1.,  1.],
    [ 1.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1.],
    [ 1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.],
    [ 1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.]
], dtype=np.int32)

rat=(0,0)
target=(9,9)

solver= BFS(maze1)
path=solver.bfs(rat,target)
print(path)
solver.render_path(path)

[(0, 0), (1, 0), (1, 1), (1, 2), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (1, 6), (2, 6), (2, 7), (3, 7), (3, 8), (3, 9), (4, 9), (5, 9), (6, 9), (6, 8), (6, 7), (6, 6), (6, 5), (7, 5), (7, 4), (7, 3), (7, 2), (7, 1), (7, 0), (8, 0), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (8, 6), (8, 7), (8, 8), (9, 8), (9, 9)]


In [16]:
import numpy as np
import cv2

class DFS:
    def __init__(self, maze):
        self.maze = maze
        self.height = len(maze)
        self.width = len(maze[0])
    
    def get_children(self, current):
        children = []
        x, y = current
        moves = [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]
        for move in moves:
            if 0 <= move[0] < self.maze.shape[0] and 0 <= move[1] < self.maze.shape[1] and self.maze[move[0], move[1]] == 1:
                children.append(move)
        return children

    def dfs(self, start, goal):
        self.rat = start
        self.target = goal
        stack = [(start, [start])]
        visited = set()
        path = []
        while stack:
            current_node, path = stack.pop() 
            if current_node == goal:
                cv2.destroyAllWindows()
                return path 
            if current_node not in visited:
                visited.add(current_node)
                children = self.get_children(current_node)
                for child in children:
                    if child not in visited:
                        stack.append((child, path + [child]))
            self.rat = current_node
            self.render()
        cv2.destroyAllWindows()
        return None
    
    def render(self, mode='human'):
        canvas = np.zeros((self.height, self.width, 3), dtype=np.uint8)
        canvas[:, :, 2] = self.maze * 255
        canvas[self.rat[0], self.rat[1], :] = [0, 255, 0]  
        canvas[self.target[0], self.target[1], :] = [255, 0, 0] 
        resized_canvas = cv2.resize(canvas, None, fx=75, fy=75, interpolation=cv2.INTER_NEAREST)
        cv2.imshow('Maze', resized_canvas)
        cv2.waitKey(50)

    def render_path(self, path):
        for node in path:
            self.rat = node
            self.render()
        cv2.destroyAllWindows()

maze1 = np.array([
    [1., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
    [1., 1., 1., 1., 1., 0., 1., 1., 1., 1.],
    [1., 1., 1., 1., 1., 0., 1., 1., 1., 1.],
    [0., 0., 1., 0., 0., 1., 0., 1., 1., 1.],
    [1., 1., 0., 1., 0., 1., 0., 0., 0., 1.],
    [1., 1., 0., 1., 0., 1., 1., 1., 1., 1.],
    [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
    [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
    [1., 0., 0., 0., 0., 0., 1., 1., 1., 1.],
    [1., 1., 1., 1., 1., 1., 1., 0., 1., 1.]
], dtype=np.int32)

rat = (0, 0)
target = (9, 9)

dfs = DFS(maze1)
path = dfs.dfs(rat, target)

print(path)
if path:
    dfs.render_path(path)


[(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 9), (1, 8), (1, 7), (1, 6), (2, 6), (2, 7), (2, 8), (2, 9), (3, 9), (4, 9), (5, 9), (5, 8), (5, 7), (5, 6), (5, 5), (6, 5), (6, 4), (6, 3), (6, 2), (6, 1), (6, 0), (7, 0), (8, 0), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (8, 6), (8, 7), (8, 8), (8, 9), (9, 9)]
